In [ ]:
import warnings
warnings.filterwarnings("ignore", category=Warning)
warnings.simplefilter(action='ignore', category=FutureWarning)
#Libraries for Data
import json
import requests 
from requests.exceptions import ReadTimeout
import time
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
#Specific NBA Libraries
from nba_api.stats.endpoints import playercareerstats
from nba_api.stats.endpoints import commonteamroster
from nba_api.stats.static import teams
from nba_api.stats.endpoints import teamestimatedmetrics

In [ ]:
#Help with API Timeouts
from nba_api.stats.endpoints import commonplayerinfo
headers = {
    #Put own headers
}

In [ ]:
#Get all teams in NBA
nba_teams = teams.get_teams()
teams = pd.DataFrame(nba_teams)

In [ ]:
#Get head to head for past 3 seasons
from nba_api.stats.endpoints import leaguegamefinder
seasons = ['2022-23','2023-24','2024-25']
matchup = []
failed = []
for year in seasons:
    for id1 in teams['id']:
        for id2 in teams['id']:
            if id1 != id2:
                try:
                    matches = leaguegamefinder.LeagueGameFinder(season_nullable = year, team_id_nullable = id1, vs_team_id_nullable=id2)
                    games = matches.get_data_frames()[0]
                    matchup.append(games)
                except ReadTimeout:
                    print(f"Timed out, appended {(id1,id2)}")
                    failed.append((id1,id2))
                except Exception as e:
                    print(f"Other error for {(id1, id2)}: {e}")
                    failed.append((id1,id2))
                time.sleep(1)

In [ ]:
#Fetch data for all failed requests until failed list is empty
while len(failed)> 0:
    current = failed.copy()
    failed.clear()  
    for id1, id2 in current:
        try:
            matches = leaguegamefinder.LeagueGameFinder(season_nullable = year, team_id_nullable = id1, vs_team_id_nullable=id2)
            games = matches.get_data_frames()[0]
            matchup.append(games)
        except ReadTimeout:
            print(f"Timed out, appended {(id1,id2)}")
            failed.append((id1,id2))
        except Exception as e:
            print(f"Other error for {(id1, id2)}: {e}")
            failed.append((id1,id2))
        time.sleep(1)

In [ ]:
#Get all head to head matchups
head_to_head = pd.concat(matchup, ignore_index = True)
head_to_head.to_csv('games_2022-25.csv')

In [ ]:
#Current season
matchupCurrent = []
failedCurrent = []
for id1 in teams['id']:
        for id2 in teams['id']:
            if id1 != id2:
                try:
                    matches = leaguegamefinder.LeagueGameFinder(season_nullable = '2025-26', team_id_nullable = id1, vs_team_id_nullable=id2)
                    games = matches.get_data_frames()[0]
                    matchupCurrent.append(games)
                except ReadTimeout:
                     print(f"Timed out, appended {(id1,id2)}")
                     failedCurrent.append((id1,id2))
                except Exception as e:
                    print(f"Other error for {(id1, id2)}: {e}")
                    failedCurrent.append((id1,id2))
                time.sleep(1)

In [ ]:
#Get all failed
while len(failedCurrent)> 0:
    for id1, id2 in failed:
        try:
            matches = leaguegamefinder.LeagueGameFinder(season_nullable = year, team_id_nullable = id1, vs_team_id_nullable=id2)
            games = matches.get_data_frames()[0]
            matchupCurrent.append(games)
        except ReadTimeout:
             print(f"Timed out, appended {(id1,id2)}")
             failedCurrent.append((id1,id2))
        except Exception as e:
            print(f"Other error for {(id1, id2)}: {e}")
            failedCurrent.append((id1,id2))
        time.sleep(1)

In [ ]:
#Get all head to head matchups
head_to_head2 = pd.concat(matchupCurrent, ignore_index = True)
head_to_head2.to_csv('games_current.csv')